In [ ]:
!pip install selenium
!pip install selenium-wire

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import threading
import time
from selenium.webdriver.chrome.options import Options
from seleniumwire import webdriver
from collections import deque
import itertools
file_name = "sample.txt"

options = {
    'proxy': {
        'https': '',
        
    }
}


proxies=deque(['Use your custom proxies here'])



# # Open the file for reading
# with open(file_name, "r") as file:
#     # Read the contents of the file
#     for line in file:
#         line=line.strip()
#         proxies.append(line)
    
proxy_cycle = itertools.cycle(proxies)
initial_proxy = next(proxy_cycle)
options['proxy']['https']='https://'+initial_proxy



def change_proxy(index):
    thread_ip = proxies[index % len(proxies)]
    tt='http://'+thread_ip
    print("Proxy used: ",tt)
    print("\n")
    options1= {
    'proxy': {
        'https': tt,
        
    }
}
    return webdriver.Chrome(seleniumwire_options=options1)


target_titles=input("Enter the title amount that needs to be scrapped for the specific url: ")
target_titles=int(target_titles)
target_uri=input("Please copy paste the url of the target ebay webpage product: ")
driver = webdriver.Chrome()

# Open a website

def Ebay_loader(target_uri):
    driver.get(target_uri)
    time.sleep(4)
    button = element = driver.find_element(By.XPATH, '/html/body/div[5]/div[4]/div[1]/div[2]/div[1]/div[3]/div[2]/span/span/button/span')
    button.click()
    time.sleep(2)
    button = element = driver.find_element(By.XPATH,'//*[@id="s0-53-17-5-4[1]-74-40-2-content-menu"]/li[2]/button/span')
    button.click()
    time.sleep(4)

    button=element = driver.find_element(By.XPATH,'//*[@id="cust-ipp-4"]')
    button.click()
    time.sleep(2)

    button=element = driver.find_element(By.XPATH,'/html/body/div[7]/div[2]/div[3]/div/form/div[2]/button[3]')
    button.click()
    time.sleep(2)
# Close the browser
#driver.quit()


    button=element = driver.find_element(By.XPATH,'/html/body/div[5]/div[4]/div[1]/div[2]/div[2]/div/div[1]/h1')
    total_results=button.text
    int_results=total_results.split()[0]
    if "+" in int_results:
        temp=int_results
        temp=temp.split("+")
        int_results=temp[0].replace(',','')
        
    int_results=int(int_results)
    int_results+=2
    
    return int_results


def next_page_shift(count,int_results,Titles_list):
    
    for i in range(2,243):
        if count<int_results:
            if i%241!=0:
            
                element = driver.find_element(By.XPATH,f"/html/body/div[5]/div[4]/div[2]/div[1]/div[2]/ul/li[{i}]/div/div[2]/a/div/span")
                Titles_list.append(element.text)
                
                count+=1
            else:
                print("\n Crawling to next page hehehe")
                element= driver.find_element(By.XPATH,"/html/body/div[5]/div[4]/div[2]/div[1]/div[2]/ul/li[242]/div[2]/span/span/nav/a")
                element.click()
            
                time.sleep(3)
                count+=1
                break
        
        else:
            break
    
    return count,Titles_list


int_results=Ebay_loader(target_uri)
count=2

Titles_list=[]
while count<int_results:
    count,Titles_List=next_page_shift(count,int_results,Titles_list)

print(len(Titles_list))

driver.quit()



# Find the search input element


threads=[]
Parent_sub_categories=[]

num_threads = 3

# Calculate the number of URLs per chunk

HREF=[]

def Threading_for_Amazon(chunk,index):
    
    chunkk=chunk
    indd=index
    
    try:
    
        for url in chunk:

            ASNN=""
            driver1 =change_proxy(indd)
            query=url+" Amazon us"
            url=f'https://www.google.com/search?q={query}'
            driver1.get(url)
            time.sleep(2)

            first_result = driver1.find_element(By.CSS_SELECTOR, 'h3')
            first_result.click()

            current_url = driver1.current_url
            current_url=current_url.split("/")
            ASN=current_url[5]
            ASN=ASN.split("?")


            Parent_ASN_List.append(ASN[0])
            print("Parent ASN: ",ASN[0])
            ASNN=ASN[0]

            span_xpath = 'a-list-item'
            span_xpath1 = 'span.a-list-item'

            span_xpath = f'//span[@class="{span_xpath}"]/a'

    # Find the first <span> element using the constructed XPath
            first_span = driver1.find_elements(By.XPATH,span_xpath)

            
            hreff=[]
            for span in first_span:
                hreff = span.get_attribute('href')
            
            
            first_span = driver1.find_elements(By.CSS_SELECTOR,span_xpath1)

            subcategory=""
            categories=[]
            for span in first_span:
            
                categories.append(span.text)
    
    
            element_to_find = '›'

# Use slicing to find the last index of 3 in the list
            try:
                last_index = len(categories) - categories[::-1].index(element_to_find) - 1
            except ValueError:
                last_index = -1  # Element not found

            subcategory=categories[last_index+1]
            print("Parent Subcategory: ", subcategory)
            Parent_sub_categories.append(subcategory)

            time.sleep(2)
            
            driver1=change_proxy(indd)
            
            print("Loading Subcategory page.....!")
            print("\n")
            
            driver1.get(hreff[len(hreff)-2])
            
            time.sleep(2)

            a_elements = driver1.find_elements("tag name",'a')

            Href=[]
            titles=[]
    # Extract titles and href attributes
            for a_element in a_elements:
                title = a_element.text
                href = a_element.get_attribute('href')
                if title and href:
                    temp=title
                    temp=temp.split(" ")
                    for name in temp:
                        if name in href:
                            Href.append(href)
                            break

                        else:
                            continue



            sub_ASIN=[]
            for url in set(Href):
                temp=url
                temp=temp.split("/")
                try:
                    ASIN=temp[5]
                    if len(ASIN)==10:
                        sub_ASIN.append(ASIN)


                except:
                    continue

            Sub_final_ASN_List.append(sub_ASIN)

            if len(sub_ASIN)>1:
                Parent_sub_ASN[ASNN]=sub_ASIN
                #rint(Parent_sub_ASN)

            else:
                print("No sufficient Barcodes to extract.....")

                driver1.quit()
        
    
    except Exception:
        driver1.quit()
        print("Expected Amazon captcha or other error, trying again!.....")
        print("\n")
        
        Threading_for_Amazon(chunkk,indd+1)
        


        
Parent_ASN_List=[]
sub_ASN_List=[]
Sub_final_ASN_List=[]
tt=[]

Parent_sub_ASN={}

# Enter your search query
threads = []

urls_per_chunk = target_titles // num_threads

for i in range(num_threads):
    updated_uri=[]
    updated_uri=Titles_list[:target_titles]
    start = i * urls_per_chunk
    end = start + urls_per_chunk if i < num_threads - 1 else None
    chunk = updated_uri[start:end]

    thread = threading.Thread(target=Threading_for_Amazon,args=(chunk,i))
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

# This part of the code will execute after all threads are done


print("\n")


keys = list(Parent_sub_ASN.keys())
values = list(Parent_sub_ASN.values())

csv_dict=[]



for i in range(len(keys)):
    key = keys[i]
    value = values[i]
    
    print(key,"   ", Parent_sub_categories[i],"   ", value)
    
    
print("All web pages have been loaded.")    
    
